In [1]:
#!/usr/bin/env python3
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter
from pandas import ExcelWriter

# === Einstellungen ===
FONT = Font(name='Arial', size=9)
FONT_BOLD = Font(name='Arial', size=9, bold=True)
thin = Side(border_style='thin', color='000000')
border = Border(left=thin, right=thin, top=thin, bottom=thin)
# Hinweis: Spalte 1=Algorithmus, 2=Model, dann Time ab Spalte 3
group_ranges = [('Time', 3, 5), ('Heap', 6, 8), ('Stack', 9, 11)]
subtitles = ['Algorithmus', 'Model', 'Key', 'Enc', 'Dec', 'Key', 'Enc', 'Dec', 'Key', 'Enc', 'Dec']

# 1. Pfade definieren
csv_input     = 'output/csv/nist_all.csv'
output_dir    = 'output/xlsx'
combined_file = os.path.join(output_dir, 'nist_all.xlsx')

# 2. Output-Verzeichnis sicherstellen
os.makedirs(output_dir, exist_ok=True)

# 3. CSV einlesen und Spaltennamen vereinheitlichen
df = pd.read_csv(csv_input)
df.columns = df.columns.str.lower()

# 4. Flaches DataFrame und Header definieren
subtitle_map = {'enc': 'Enc', 'dec': 'Dec'}
# Jetzt mit Algorithmus und Model
df_flat = pd.DataFrame({
    'algorithmus':  df['algorithmus'],
    'model':        df['model'],
    'keygen_time':  df['keygen_time'],
    'enc_time':     df['enc_time'],
    'dec_time':     df['dec_time'],
    'keygen_heap':  df['keygen_heap'],
    'enc_heap':     df['enc_heap'],
    'dec_heap':     df['dec_heap'],
    'keygen_stack': df['keygen_stack'],
    'enc_stack':    df['enc_stack'],
    'dec_stack':    df['dec_stack'],
})
col_labels = ['Algorithmus', 'Model', 'Key', subtitle_map['enc'], subtitle_map['dec'],
              'Key', subtitle_map['enc'], subtitle_map['dec'],
              'Key', subtitle_map['enc'], subtitle_map['dec']]
df_flat.columns = col_labels

# 5. Kombinierte Datei schreiben
with ExcelWriter(combined_file, engine='openpyxl') as writer:
    df_flat.to_excel(writer, index=False, header=False, sheet_name='NIST All', startrow=2)
    wb = writer.book
    ws = wb['NIST All']

    # Runden-Wert in A1
    runden_set  = df['runden'].dropna().unique()
    round_value = int(runden_set[0]) if len(runden_set) == 1 else 'Var'
    c = ws.cell(row=1, column=1, value=f"R = {round_value}")
    c.font = FONT
    c.alignment = Alignment(horizontal='center', vertical='center')
    c.border    = border

    # Header-Gruppen
    for title, start, end in group_ranges:
        ws.merge_cells(start_row=1, start_column=start, end_row=1, end_column=end)
        c = ws.cell(row=1, column=start)
        c.value     = title
        c.font      = FONT_BOLD
        c.alignment = Alignment(horizontal='center', vertical='center')
        c.border    = border

    # Untertitelreihe
    for idx, sub in enumerate(subtitles, start=1):
        c = ws.cell(row=2, column=idx, value=sub)
        c.font      = FONT_BOLD
        c.alignment = Alignment(horizontal='center', vertical='center')
        c.border    = border

    # Datenformatierung
    for row in ws.iter_rows(min_row=3, max_row=ws.max_row, min_col=1, max_col=11):
        for cell in row:
            cell.font   = FONT
            cell.border = border

    # Spaltenbreiten anpassen
    for col in range(1, len(col_labels) + 1):
        letter = get_column_letter(col)
        max_len = max((len(str(c.value)) for c in ws[letter] if c.value is not None), default=0)
        ws.column_dimensions[letter].width = max_len + 2

print(f"Kombinierte Datei erstellt: {combined_file}")

Kombinierte Datei erstellt: output/xlsx\nist_all.xlsx
